In [1]:
### Reading the data
import json
import os
data = {}
dirname = "./workshop_dataset1"
for filename in os.listdir(dirname):
	with open(dirname+'/'+filename) as f:
		data.update(json.load(f))
		f.close()

In [2]:
# data = json.load(open("./workshop_dataset1/2017-11-26.json"))

In [3]:
# data

In [4]:
# [data[d]["datetime"] for d in data]

In [5]:
from py2neo import Graph, Node, Relationship
g = Graph("bolt://localhost:7687", password="pass")
tx = g.begin()

In [6]:
for d in data:
	u = Node("User",
			 # verified=data[d]["verified"],
			 # author=data[d]["author"],
			 # author_profile_image=data[d]["author_profile_image"],
			 author_screen_name=data[d]["author_screen_name"],
			 # author_id=data[d]["author_id"]
			 )
	tx.merge(u, primary_label="User", primary_key="author_screen_name")
	
	u["verified"] = data[d]["verified"]
	u["author"] = data[d]["author"]
	u["author_profile_image"] = data[d]["author_profile_image"]
	u["author_id"] = data[d]["author_id"]
	
	# u.push()
	tx.push(u)
	
	t = Node("Tweet",
			 # quote_count=data[d]["quote_count"],
			 # reply_count=data[d]["reply_count"],
			 # datetime=data[d]["datetime"],
			 # date=data[d]["date"],
			 # like_count=data[d]["like_count"],
			 # sentiment=data[d]["sentiment"],
			 # location=data[d]["location"],
			 tid=data[d]["tid"],
			 # retweet_count=data[d]["retweet_count"],
			 # ttype=data[d]["type"],
			 # tweet_text=data[d]["tweet_text"],
			 # lang=data[d]["lang"]
			 )
	tx.merge(t, primary_label="Tweet", primary_key="tid")
	# tx.create(t)
	
	t["quote_count"] = data[d]["quote_count"]
	t["reply_count"] = data[d]["reply_count"]
	t["datetime"] = data[d]["datetime"]
	t["date"] = data[d]["date"]
	t["like_count"] = data[d]["like_count"]
	t["sentiment"] = data[d]["sentiment"]
	t["location"] = data[d]["location"]
	t["retweet_count"] = data[d]["retweet_count"]
	t["ttype"] = data[d]["type"]
	t["tweet_text"] = data[d]["tweet_text"]
	t["lang"] = data[d]["lang"]
	
	# t.push()
	tx.push(t)

	POSTS = Relationship(u, "POSTS", t)

	tx.create(POSTS)
	
	if data[d]["quoted_source_id"]:
		quoted = Node("Tweet",
					 tid=data[d]["quoted_source_id"]
					 )
		tx.merge(quoted, primary_label="Tweet", primary_key="tid")

		QUOTES = Relationship(t, "QUOTES", quoted)

		tx.create(QUOTES)
	
	if data[d]["retweet_source_id"]:
		retweet = Node("Tweet",
					   tid=data[d]["retweet_source_id"]
					   )
		tx.merge(retweet, primary_label="Tweet", primary_key="tid")

		RETWEETS = Relationship(t, "RETWEETS", retweet)

		tx.create(RETWEETS)
	
	if data[d]["replyto_source_id"]:
		replyto = Node("Tweet",
					   tid=data[d]["replyto_source_id"]
					   )
		tx.merge(replyto, primary_label="Tweet", primary_key="tid")

		REPLY = Relationship(t, "REPLY", replyto)

		tx.create(REPLY)
	
	if (data[d]["keywords_processed_list"]
		and
		data[d]["keywords_processed_list"]!='NULL'
		and
		data[d]["keywords_processed_list"]!="''"
		and
		data[d]["keywords_processed_list"]!='""'
		):
		for k in data[d]["keywords_processed_list"]:
			if (k and k!='NULL' and k!='""' and k!="''"):
				keyword = Node("Keyword", keyword=k)
				
				tx.merge(keyword)

				HAS = Relationship(t, "HAS", keyword)

				tx.create(HAS)

	if (data[d]["hashtags"]
		and
		data[d]["hashtags"]!='NULL'
		and
		data[d]["hashtags"]!="''"
		and
		data[d]["hashtags"]!='""'
		):
		for h in data[d]["hashtags"]:
			if (h and h!='NULL' and h!='""' and h!="''"):
				hashtag = Node("Hashtag", hashtag=h)
				
				tx.merge(hashtag)

				TAGS = Relationship(t, "TAGS", hashtag)

				tx.create(TAGS)

	if (data[d]["mentions"]
		and
		data[d]["mentions"]!='NULL'
		and
		data[d]["mentions"]!="''"
		and
		data[d]["mentions"]!='""'
		):
		for m in data[d]["mentions"]:
			if (m and m!='NULL' and m!='""' and m!="''"):
				mention = Node("User", author_screen_name=m)
				
				tx.merge(mention, primary_label="User", primary_key="author_screen_name")

				MENTIONS = Relationship(t, "MENTIONS", mention)

				tx.create(MENTIONS)

	print(".", end="")
print("\nDONE.")

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

KeyboardInterrupt: 

Exception ignored in: 'neo4j.bolt._io.ChunkedInputBuffer.receive'
KeyboardInterrupt


ServiceUnavailable: Failed to read from defunct connection Address(host='127.0.0.1', port=7687)

In [ ]:
tx.commit()

TypeError: 'Transaction' object is not iterable